In [1]:
from keras.datasets import mnist
import numpy as np

##As previous functions was too simple to construct NN, let's construct classes for simple usage

In [2]:
class GradientDescent():


    def __init__(self, lr = 1e-3, eps = 1e-4):
        self.lr = lr
        self.eps = eps
        self.delta = 0


    def optimize(self, target, gradients):
        optimized = []
        for t, grad in zip(target, gradients):
            optimized.append(t - self.lr * grad)
            self.delta += self.lr * np.linalg.norm(grad)
        return optimized


    def stop(self):
        return not(self.delta > 1e-9 or self.delta < self.eps)

In [3]:
class Node:


    def __init__(self, input_dim, output_dim, inner_ndim):

        self.n_input = (input_dim, ) if isinstance(input_dim, int) else tuple(input_dim)
        self.input_dim = 1 if isinstance(input_dim, int) else len(self.n_input)
        self.n_output = (output_dim, ) if isinstance(output_dim, int) else tuple(output_dim)
        self.output_dim = 1 if isinstance(input_dim, int) else len(self.n_output)
        self.inner_dim = inner_ndim
        self.input = None
        self.labels = None



    def change_dims(self, x, dim):
        return np.reshape(x, x.shape[-dim:]) if x.ndim > dim else (x if x.ndim == dim else np.expand_dims(x, tuple(range(dim - x.ndim))))

In [4]:
class SoftmaxLoss(Node):


    def __init__(self, n_input):
        super().__init__(n_input, 1, 2)


    def softmax_call(self, x):
        self.sm_max_index = np.argmax(np.abs(x), axis=1).reshape(-1, 1)
        self.softmax_x_norm = x / np.max(np.abs(x), axis=1).reshape(-1, 1)
        exp = np.exp(self.softmax_x_norm)
        return exp / exp.sum(axis=1).reshape(-1, 1)


    def softmax_jacobian(self, x):
        rows, classes = self.softmax_x_norm.shape
        exp_x = np.exp(self.softmax_x_norm)
        exp_sum = exp_x.sum(axis=1)

        softmax_jacobian = np.zeros((rows, classes, classes))

        for row in range(rows):
            exp_x_row = exp_x[row]
            exp_sum_row = exp_sum[row]
            diag = np.diag([exp_xi / exp_sum_row - (exp_xi / exp_sum_row) ** 2 for exp_xi in exp_x_row])
            triag = np.array([[-exp_x_row[i] * exp_x_row[j] / exp_sum_row**2 if i > j else 0 for i in range(classes)] for j in range(classes)])
            softmax_jacobian[row] = diag + triag + triag.T

        for row in range(x.shape[0]):
            max_index = self.sm_max_index[row][0]
            x_max = np.abs(x)[row, max_index]
            dx_norm_dx = np.diag([1/x_max for _ in range(x.shape[1])])
            dx_norm_dx[:, max_index] = np.array([-x_i / x_max**2 for x_i in x[row]])
            dx_norm_dx[max_index] = np.zeros(x.shape[1])
            softmax_jacobian[row] = softmax_jacobian[row] @ dx_norm_dx

        return softmax_jacobian


    def jacobian(self, x):
        softmax_jac = self.softmax_jacobian(x)
        jac = np.zeros_like(x)
        for i in range(x.shape[0]):
            jac[i] = - self.labels[i] * softmax_jac[i].diagonal() / self.loss_elementwise[i]
        return jac


    def forward(self, input, labels = None):
        self.input = self.change_dims(input, self.inner_dim)
        self.labels = self.change_dims(labels, self.inner_dim)
        self.loss_elementwise = np.array([-np.log(self.labels[i].dot(self.softmax_call(self.input)[i])) for i in range(self.input.shape[0])])
        return self.change_dims(np.sum(self.loss_elementwise) / self.input.shape[0], self.output_dim)


    def backward(self):
        backprop_pd = self.jacobian(self.input)
        self.input = None
        self.labels = None
        return self.change_dims(backprop_pd, self.input_dim)

In [5]:
class ReLU(Node):


    def __init__(self, n_input: int):
        super().__init__(n_input, n_input, 2)


    def jacobian(self, x):
        return np.array([np.diag([1 if x_elem > 0 else 0 for x_elem in x_row]) for x_row in x])


    def forward(self, input, labels = None):
        self.input = self.change_dims(input, self.inner_dim)
        return self.change_dims(np.maximum(self.input, 0), self.output_dim)


    def backward(self, input_pd):
        input_pd = self.change_dims(input_pd, self.inner_dim)
        jacobian = self.jacobian(self.input)
        backprop_pd = np.array([jacobian[i] @ input_pd[i] for i in range(jacobian.shape[0])])
        return self.change_dims(backprop_pd, self.output_dim)


    def optimize_weights(self, optimizer):
        pass

In [6]:
class Linear(Node):


    def __init__(self, input_dim, output_dim, W = None):
        super().__init__(input_dim, output_dim, inner_ndim=2)
        self.W_dim = (self.n_input[0]+1, self.n_output[0]) if self.input_dim == 1 else (self.n_input[1]+1, self.n_output[1])
        self.W = np.random.uniform(0.4, 0.6, self.W_dim) if W is None else W
        self.input = None
        self.W_pd = None


    def jacobian(self, X):
        if self.input_dim == 1:
            jac_dim = (1, self.n_input[0], 1, self.n_output[0])
        else:
            jac_dim = (*self.n_input, *self.n_output)

        jac = np.zeros(jac_dim, dtype=np.float64)

        for i in range(jac_dim[0]):
            for j in range(jac_dim[1]):
                jac[i, j, i] = self.W[j+1]
        return jac


    def W_jacobian(self, X):
        if self.output_dim == 1:
            jac_dim = (*self.W_dim, 1, self.n_output[0])
        else:
            jac_dim = (*self.W_dim, *self.n_output)

        jac = np.zeros(jac_dim, dtype=np.float64)
        for i in range(jac_dim[0]):
            for j in range(jac_dim[1]):
                jac[i, j, :, i] = X[:, i]
        return jac


    def forward(self, input):
        self.input = self.change_dims(input, self.inner_dim)
        self.input = np.concatenate([np.ones((self.input.shape[0], 1)), self.input], axis=1)
        return self.change_dims(self.change_dims(self.input @ self.W, self.output_dim), self.output_dim)


    def backward(self, input_pd):
        input_pd = self.change_dims(input_pd, self.inner_dim)
        input_pd = input_pd.astype(np.float64)
        self.W_pd = self.input.T.dot(input_pd)
        return self.change_dims(input_pd.dot(self.W[1:, :].T), self.output_dim)


    def optimize_weights(self, gd):
        self.W = gd.optimize([self.W], [self.W_pd])[0]

In [7]:
class SoftmaxNN:


    def __init__(self, n_input, n_output, batch_size, lr):
        self.layers = [Linear((batch_size, n_input), (batch_size, n_output))]
        self.loss = SoftmaxLoss((batch_size, n_output))
        self.gd = GradientDescent(lr)


    def fit(self, X, y, n_epochs):
        n = 0
        while True:
            loss = 0
            for batch in range(X.shape[0]):
                state = X[batch]
                label = y[batch]

                state = self.predict(state)
                loss += self.loss.forward(state, label)

                upstream = self.loss.backward()
                for layer in self.layers[::-1]:
                    upstream = layer.backward(upstream)
                    layer.optimize_weights(self.gd)
            n += 1
            print(f"Epoch {n}, Loss: {loss}")

            if n >= n_epochs or self.gd.stop():
                break


    def predict(self, x):
        state = x.copy()
        for layer in self.layers:
            state = layer.forward(state)
        return state

# Softmax NN

In [8]:
def label_vec_func(labels):
    labels_matrix = np.zeros([len(labels), 10])
    labels_matrix[np.arange(len(labels)), labels] = 1
    return labels_matrix

In [9]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

n_input, n_output, batch_size = 784, 10, 2000

X_train = X_train.reshape(X_train.shape[0] // batch_size, batch_size, -1)
y_train_one_hot = label_vec_func(y_train).reshape((y_train.shape[0] // batch_size, batch_size, 10))

11490434/11490434 [==============================] - 0s 0us/step


In [10]:
network = SoftmaxNN(n_input, n_output, batch_size, 0.5)
network.fit(X_train, y_train_one_hot, n_epochs=7)

Epoch 1, Loss: [66.37385665]
Epoch 2, Loss: [64.83701333]
Epoch 3, Loss: [64.35943009]
Epoch 4, Loss: [64.09839416]
Epoch 5, Loss: [63.9277188]
Epoch 6, Loss: [63.80497339]
Epoch 7, Loss: [63.7112725]


In [11]:
def compute_accuracy(X_test, y_test, model):

    correct_predictions = 0
    total = 0

    for input, label in zip(X_test, y_test):
        predicts = model.predict(input)
        correct_predictions += (np.argmax(predicts, axis=1) == label).sum()
        total += len(label)

    return correct_predictions / total

In [12]:
X_batches = X_test.reshape((X_test.shape[0] // batch_size, batch_size, -1))
y_batches = y_test.reshape((y_test.shape[0] // batch_size, batch_size,))

In [13]:
print(f"Accuracy: {compute_accuracy(X_batches, y_batches, network)}")

Accuracy: 0.7754


#2 layer NN

In [14]:
class ReLUSoftmaxNN:


    def __init__(self, n_input, n_output, batch_size, lr):
        self.layers = [Linear((batch_size, X_train.shape[2]), (batch_size, 64)), ReLU((batch_size, 64)), Linear((batch_size, 64), (batch_size, 10))]
        self.loss = SoftmaxLoss((batch_size, n_output))
        self.gd = GradientDescent(lr)


    def fit(self, X, y, n_epochs):
        n = 0
        while True:
            loss = 0
            for batch in range(X.shape[0]):
                state = X[batch]
                label = y[batch]

                state = self.predict(state)
                loss += self.loss.forward(state, label)

                upstream = self.loss.backward()
                for layer in self.layers[::-1]:
                    upstream = layer.backward(upstream)
                    layer.optimize_weights(self.gd)
            n += 1
            print(f"Epoch {n}, Loss: {loss}")

            if n >= n_epochs or self.gd.stop():
                break


    def predict(self, x):
        state = x.copy()
        for layer in self.layers:
            state = layer.forward(state)
        return state

In [15]:
network = ReLUSoftmaxNN(784, 10, batch_size, 0.1)
network.fit(X_train, y_train_one_hot, n_epochs=7)

Epoch 1, Loss: [69.06052055]
Epoch 2, Loss: [69.04319154]
Epoch 3, Loss: [69.0405569]
Epoch 4, Loss: [69.04039233]
Epoch 5, Loss: [69.04106566]
Epoch 6, Loss: [69.04208959]
Epoch 7, Loss: [69.04327179]


In [16]:
print(f"Accuracy: {compute_accuracy(X_batches, y_batches, network)}")

Accuracy: 0.1135
